# Assignment 12: Data Preprocessing and Feature Engineering

## Dataset: Adult Census Income
Predicts whether income exceeds $50K/yr based on census data.

**Topics Covered:**
- Data Exploration and Preprocessing
- Scaling (Standard & Min-Max)
- Encoding (One-Hot & Label)
- Feature Engineering
- Feature Selection (Isolation Forest, PPS)

---
## Step 1: Import Libraries and Load Data

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import IsolationForest

# Load the dataset
df = pd.read_csv('datasets/adult_with_headers.csv')

# Display basic info
print("Dataset loaded successfully!")
print("Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

Dataset loaded successfully!
Shape: (1000, 15)

First 5 rows:


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,22,Federal-gov,1015558,Masters,13,Married-civ-spouse,Exec-managerial,Not-in-family,Black,Male,68000,0,47,Philippines,<=50K
1,53,State-gov,511907,Masters,9,Divorced,Sales,Other-relative,Black,Female,94000,200,21,Philippines,>50K
2,78,Self-emp-inc,1370445,HS-grad,9,Widowed,Machine-op-inspct,Husband,Black,Female,16000,3000,42,Philippines,<=50K
3,89,Federal-gov,155060,HS-grad,9,Widowed,Craft-repair,Other-relative,Amer-Indian-Eskimo,Female,80000,0,70,Philippines,>50K
4,76,Without-pay,629017,Doctorate,2,Separated,Machine-op-inspct,Other-relative,White,Male,0,0,89,Mexico,<=50K


---
## Task 1: Data Exploration and Preprocessing

In [3]:
# Basic data exploration
print("=== Data Types ===")
print(df.dtypes)

print("\n=== Summary Statistics ===")
df.describe()

=== Data Types ===
age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
dtype: object

=== Summary Statistics ===


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000
mean,52.787000,7.416421e+05,7.838000,27323.000000,880.400000,48.568000
std,21.370829,4.301271e+05,4.210164,32745.365014,1471.339757,27.800256
min,17.000000,1.057100e+04,1.000000,0.000000,0.000000,1.000000
25%,34.000000,3.840932e+05,4.000000,0.000000,0.000000,24.000000
50%,52.000000,7.279875e+05,8.000000,7000.000000,0.000000,49.000000
75%,72.000000,1.113147e+06,11.000000,55000.000000,1500.000000,71.000000
max,89.000000,1.492962e+06,15.000000,99000.000000,4900.000000,98.000000


In [4]:
# Check for missing values
print("=== Missing Values ===")

# In this dataset, missing values are represented as ' ?' (with space)
# Let's check for this pattern
for column in df.columns:
    # Check for ' ?' pattern
    if df[column].dtype == 'object':
        missing_count = (df[column].str.strip() == '?').sum()
        if missing_count > 0:
            print(column + ":", missing_count, "missing values")

# Also check for regular null values
print("\nNull values:")
print(df.isnull().sum())

=== Missing Values ===

Null values:
age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64


In [5]:
# Handle missing values (replace '?' with NaN then fill with mode)
print("=== Handling Missing Values ===")

# Clean string columns by stripping whitespace
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.strip()
        # Replace '?' with NaN
        df[column] = df[column].replace('?', np.nan)

# Fill missing values with mode (most frequent value)
for column in df.columns:
    missing_count = df[column].isnull().sum()
    if missing_count > 0:
        mode_value = df[column].mode()[0]
        df[column] = df[column].fillna(mode_value)
        print("Filled", missing_count, "missing values in", column, "with mode:", mode_value)

print("\nRemaining missing values:", df.isnull().sum().sum())

=== Handling Missing Values ===

Remaining missing values: 0


### 1.1 Scaling Techniques

**Standard Scaling:** Transforms data to have mean=0 and std=1
- Best when: Data is normally distributed, algorithm uses distance measures

**Min-Max Scaling:** Transforms data to range [0, 1]
- Best when: You need bounded values, neural networks

In [6]:
# Identify numerical columns
numerical_columns = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

print("Original Data (first 5 rows):")
print(df[numerical_columns].head())

Original Data (first 5 rows):
   age   fnlwgt  education_num  capital_gain  capital_loss  hours_per_week
0   22  1015558             13         68000             0              47
1   53   511907              9         94000           200              21
2   78  1370445              9         16000          3000              42
3   89   155060              9         80000             0              70
4   76   629017              2             0             0              89


In [7]:
# Standard Scaling
print("=== Standard Scaling ===")

# Create a copy for standard scaling
df_standard = df.copy()

# Apply standard scaling manually (to show the process)
for column in numerical_columns:
    mean_val = df[column].mean()
    std_val = df[column].std()
    df_standard[column + '_std'] = (df[column] - mean_val) / std_val

print("After Standard Scaling:")
std_cols = [col + '_std' for col in numerical_columns]
print(df_standard[std_cols].head())

print("\nVerification (mean should be ~0, std should be ~1):")
for col in std_cols:
    print(col, "- Mean:", round(df_standard[col].mean(), 4), "Std:", round(df_standard[col].std(), 4))

=== Standard Scaling ===
After Standard Scaling:
    age_std  fnlwgt_std  education_num_std  capital_gain_std  \
0 -1.440609    0.636825           1.226080          1.242222   
1  0.009967   -0.534110           0.275999          2.036227   
2  1.179786    1.461900           0.275999         -0.345789   
3  1.694506   -1.363741           0.275999          1.608686   
4  1.086200   -0.261841          -1.386644         -0.834408   

   capital_loss_std  hours_per_week_std  
0         -0.598366           -0.056402  
1         -0.462436           -0.991646  
2          1.440592           -0.236257  
3         -0.598366            0.770928  
4         -0.598366            1.454375  

Verification (mean should be ~0, std should be ~1):
age_std - Mean: 0.0 Std: 1.0
fnlwgt_std - Mean: 0.0 Std: 1.0
education_num_std - Mean: -0.0 Std: 1.0
capital_gain_std - Mean: -0.0 Std: 1.0
capital_loss_std - Mean: 0.0 Std: 1.0
hours_per_week_std - Mean: 0.0 Std: 1.0


In [8]:
# Min-Max Scaling
print("=== Min-Max Scaling ===")

# Create a copy for min-max scaling
df_minmax = df.copy()

# Apply min-max scaling manually
for column in numerical_columns:
    min_val = df[column].min()
    max_val = df[column].max()
    df_minmax[column + '_mm'] = (df[column] - min_val) / (max_val - min_val)

print("After Min-Max Scaling:")
mm_cols = [col + '_mm' for col in numerical_columns]
print(df_minmax[mm_cols].head())

print("\nVerification (min should be 0, max should be 1):")
for col in mm_cols:
    print(col, "- Min:", round(df_minmax[col].min(), 4), "Max:", round(df_minmax[col].max(), 4))

=== Min-Max Scaling ===
After Min-Max Scaling:
     age_mm  fnlwgt_mm  education_num_mm  capital_gain_mm  capital_loss_mm  \
0  0.069444   0.677950          0.857143         0.686869         0.000000   
1  0.500000   0.338194          0.571429         0.949495         0.040816   
2  0.847222   0.917352          0.571429         0.161616         0.612245   
3  1.000000   0.097470          0.571429         0.808081         0.000000   
4  0.819444   0.417195          0.071429         0.000000         0.000000   

   hours_per_week_mm  
0           0.474227  
1           0.206186  
2           0.422680  
3           0.711340  
4           0.907216  

Verification (min should be 0, max should be 1):
age_mm - Min: 0.0 Max: 1.0
fnlwgt_mm - Min: 0.0 Max: 1.0
education_num_mm - Min: 0.0 Max: 1.0
capital_gain_mm - Min: 0.0 Max: 1.0
capital_loss_mm - Min: 0.0 Max: 1.0
hours_per_week_mm - Min: 0.0 Max: 1.0


**When to use which scaling:**

| Standard Scaling | Min-Max Scaling |
|-----------------|----------------|
| Data is normally distributed | Data needs to be in [0,1] range |
| Algorithms: SVM, Logistic Regression | Algorithms: Neural Networks, KNN |
| Not affected by outliers much | Sensitive to outliers |

---
## Task 2: Encoding Techniques

In [9]:
# Identify categorical columns
categorical_columns = ['workclass', 'education', 'marital_status', 'occupation', 
                       'relationship', 'race', 'sex', 'native_country', 'income']

# Count unique values in each categorical column
print("=== Unique Values in Categorical Columns ===")
for column in categorical_columns:
    unique_count = df[column].nunique()
    print(column + ":", unique_count, "unique values")

=== Unique Values in Categorical Columns ===
workclass: 7 unique values
education: 7 unique values
marital_status: 5 unique values
occupation: 6 unique values
relationship: 6 unique values
race: 5 unique values
sex: 2 unique values
native_country: 6 unique values
income: 2 unique values


In [10]:
# Separate columns based on unique values
# Less than 5 categories -> One-Hot Encoding
# 5 or more categories -> Label Encoding

one_hot_columns = []  # < 5 categories
label_encode_columns = []  # >= 5 categories

for column in categorical_columns:
    unique_count = df[column].nunique()
    if unique_count < 5:
        one_hot_columns.append(column)
    else:
        label_encode_columns.append(column)

print("One-Hot Encoding columns (< 5 categories):")
print(one_hot_columns)

print("\nLabel Encoding columns (>= 5 categories):")
print(label_encode_columns)

One-Hot Encoding columns (< 5 categories):
['sex', 'income']

Label Encoding columns (>= 5 categories):
['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'native_country']


In [11]:
# One-Hot Encoding
print("=== One-Hot Encoding ===")

df_encoded = df.copy()

# Apply one-hot encoding using pandas get_dummies
df_encoded = pd.get_dummies(df_encoded, columns=one_hot_columns)

print("Original shape:", df.shape)
print("After One-Hot Encoding:", df_encoded.shape)

# Show new columns created
new_cols = [col for col in df_encoded.columns if col not in df.columns]
print("\nNew columns created:")
for col in new_cols[:10]:
    print(" -", col)

=== One-Hot Encoding ===
Original shape: (1000, 15)
After One-Hot Encoding: (1000, 17)

New columns created:
 - sex_Female
 - sex_Male
 - income_<=50K
 - income_>50K


In [12]:
# Label Encoding
print("=== Label Encoding ===")

# Apply label encoding
for column in label_encode_columns:
    print("\nEncoding:", column)
    print("Original values:", df_encoded[column].unique()[:5], "...")
    
    # Create mapping dictionary
    unique_values = df_encoded[column].unique()
    mapping = {}
    for i in range(len(unique_values)):
        mapping[unique_values[i]] = i
    
    # Apply mapping
    df_encoded[column] = df_encoded[column].map(mapping)
    print("Encoded values:", df_encoded[column].unique()[:5], "...")

print("\nLabel Encoding complete!")

=== Label Encoding ===

Encoding: workclass
Original values: ['Federal-gov' 'State-gov' 'Self-emp-inc' 'Without-pay' 'Local-gov'] ...
Encoded values: [0 1 2 3 4] ...

Encoding: education
Original values: ['Masters' 'HS-grad' 'Doctorate' 'Some-college' 'Bachelors'] ...
Encoded values: [0 1 2 3 4] ...

Encoding: marital_status
Original values: ['Married-civ-spouse' 'Divorced' 'Widowed' 'Separated' 'Never-married'] ...
Encoded values: [0 1 2 3 4] ...

Encoding: occupation
Original values: ['Exec-managerial' 'Sales' 'Machine-op-inspct' 'Craft-repair'
 'Tech-support'] ...
Encoded values: [0 1 2 3 4] ...

Encoding: relationship
Original values: ['Not-in-family' 'Other-relative' 'Husband' 'Unmarried' 'Own-child'] ...
Encoded values: [0 1 2 3 4] ...

Encoding: race
Original values: ['Black' 'Amer-Indian-Eskimo' 'White' 'Other' 'Asian-Pac-Islander'] ...
Encoded values: [0 1 2 3 4] ...

Encoding: native_country
Original values: ['Philippines' 'Mexico' 'Canada' 'India' 'United-States'] ...
Encode

**Pros and Cons:**

| One-Hot Encoding | Label Encoding |
|-----------------|----------------|
| **Pros:** No ordinal assumption | **Pros:** Simple, no extra columns |
| **Pros:** Works with all algorithms | **Pros:** Memory efficient |
| **Cons:** Creates many columns | **Cons:** Implies ordinal relationship |
| **Cons:** Memory intensive | **Cons:** May mislead some algorithms |

---
## Task 3: Feature Engineering

In [13]:
# Create new features
print("=== Feature Engineering ===")

# Feature 1: Total Capital (gain - loss)
# Rationale: Net capital change is more informative than separate gain/loss
df['total_capital'] = df['capital_gain'] - df['capital_loss']
print("Feature 1: total_capital = capital_gain - capital_loss")
print("This shows the net financial impact of capital transactions.")
print("Sample values:", df['total_capital'].head().tolist())

print()

=== Feature Engineering ===
Feature 1: total_capital = capital_gain - capital_loss
This shows the net financial impact of capital transactions.
Sample values: [68000, 93800, 13000, 80000, 0]



In [14]:
# Feature 2: Age Group
# Rationale: Income patterns often differ by age brackets
print("Feature 2: age_group")
print("Rationale: Income patterns often differ by life stages.")

# Create age groups using simple if-else logic
age_groups = []
for age in df['age']:
    if age < 25:
        age_groups.append('Young')
    elif age < 45:
        age_groups.append('Middle')
    elif age < 65:
        age_groups.append('Senior')
    else:
        age_groups.append('Elder')

df['age_group'] = age_groups
print("Age groups created:")
print(df['age_group'].value_counts())

Feature 2: age_group
Rationale: Income patterns often differ by life stages.
Age groups created:
age_group
Elder     338
Middle    282
Senior    261
Young     119
Name: count, dtype: int64


In [ ]:
# Log Transformation on skewed feature
print("=== Log Transformation ===")

# Check skewness of capital_gain
print("Checking skewness of capital_gain:")
print("Mean:", df['capital_gain'].mean())
print("Median:", df['capital_gain'].median())
print("Skewness:", df['capital_gain'].skew())

# The large difference between mean and median indicates right skewness

# Apply log transformation (adding 1 to avoid log(0))
df['capital_gain_log'] = np.log1p(df['capital_gain'])

print("\nAfter log transformation:")
print("New skewness:", df['capital_gain_log'].skew())

# Visualize before and after
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df['capital_gain'], bins=50, color='blue', edgecolor='black')
axes[0].set_title('Before: capital_gain')
axes[0].set_xlabel('Value')

axes[1].hist(df['capital_gain_log'], bins=50, color='green', edgecolor='black')
axes[1].set_title('After: capital_gain_log')
axes[1].set_xlabel('Log Value')

plt.tight_layout()
plt.show()

---
## Task 4: Feature Selection

### 4.1 Isolation Forest for Outlier Detection

**Isolation Forest** identifies outliers by isolating observations.
Outliers are easier to isolate and thus get shorter path lengths.

In [16]:
# Isolation Forest for Outlier Detection
print("=== Isolation Forest ===")

# Select numerical features for outlier detection
features_for_outliers = df[numerical_columns].copy()

# Create Isolation Forest model
iso_forest = IsolationForest(contamination=0.05, random_state=42)

# Fit and predict
outlier_labels = iso_forest.fit_predict(features_for_outliers)

# Count outliers (labeled as -1)
outliers_count = 0
inliers_count = 0
for label in outlier_labels:
    if label == -1:
        outliers_count = outliers_count + 1
    else:
        inliers_count = inliers_count + 1

print("Total records:", len(outlier_labels))
print("Outliers detected:", outliers_count)
print("Inliers (normal):", inliers_count)
print("Outlier percentage:", round(outliers_count/len(outlier_labels)*100, 2), "%")

=== Isolation Forest ===
Total records: 1000
Outliers detected: 50
Inliers (normal): 950
Outlier percentage: 5.0 %


In [17]:
# Create dataframe without outliers
df['is_outlier'] = outlier_labels
df_clean = df[df['is_outlier'] == 1].copy()

print("Original dataset size:", len(df))
print("After removing outliers:", len(df_clean))

print("\n=== How Outliers Affect Model Performance ===")
print("1. Outliers can skew model training towards extreme values")
print("2. Distance-based algorithms (KNN, SVM) are especially sensitive")
print("3. Outliers can increase model variance and reduce generalization")
print("4. They may represent data errors or rare but valid observations")

Original dataset size: 1000
After removing outliers: 950

=== How Outliers Affect Model Performance ===
1. Outliers can skew model training towards extreme values
2. Distance-based algorithms (KNN, SVM) are especially sensitive
3. Outliers can increase model variance and reduce generalization
4. They may represent data errors or rare but valid observations


### 4.2 Correlation Matrix

In [ ]:
# Correlation Matrix
print("=== Correlation Matrix ===")

# Select numerical columns for correlation
numerical_data = df[numerical_columns + ['total_capital', 'capital_gain_log']]

# Calculate correlation
correlation_matrix = numerical_data.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

### 4.3 PPS (Predictive Power Score)

**PPS** measures how well one variable can predict another.
Unlike correlation, it:
- Works with categorical variables
- Captures non-linear relationships
- Is asymmetric (A->B may differ from B->A)

In [19]:
# Simple PPS-like analysis using cross-tabulation
# Note: Full PPS requires 'ppscore' library
print("=== Predictive Power Analysis ===")

# Calculate predictive relationships for categorical variables
# Using chi-square test approach

from scipy.stats import chi2_contingency

# Check relationship between education and income
print("\nEducation vs Income:")
contingency_table = pd.crosstab(df['education'], df['income'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
print("Chi-square statistic:", round(chi2, 2))
print("P-value:", p_value)
if p_value < 0.05:
    print("Result: Strong relationship exists!")

# Check relationship between occupation and income
print("\nOccupation vs Income:")
contingency_table = pd.crosstab(df['occupation'], df['income'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
print("Chi-square statistic:", round(chi2, 2))
print("P-value:", p_value)
if p_value < 0.05:
    print("Result: Strong relationship exists!")

=== Predictive Power Analysis ===

Education vs Income:
Chi-square statistic: 3.41
P-value: 0.7560921734230491

Occupation vs Income:
Chi-square statistic: 6.46
P-value: 0.2641815813220281


In [20]:
# Compare findings
print("=== Comparison: Correlation vs PPS ===")
print()
print("Correlation Matrix:")
print("- Only measures LINEAR relationships")
print("- Works only with numerical variables")
print("- Symmetric (A,B) = (B,A)")
print()
print("PPS (Predictive Power Score):")
print("- Measures ANY relationship (linear or non-linear)")
print("- Works with categorical AND numerical variables")
print("- Asymmetric (A predicting B may differ from B predicting A)")
print()
print("Key Finding: Variables like education and occupation")
print("show strong predictive power for income even though")
print("they can't be measured using simple correlation.")

=== Comparison: Correlation vs PPS ===

Correlation Matrix:
- Only measures LINEAR relationships
- Works only with numerical variables
- Symmetric (A,B) = (B,A)

PPS (Predictive Power Score):
- Measures ANY relationship (linear or non-linear)
- Works with categorical AND numerical variables
- Asymmetric (A predicting B may differ from B predicting A)

Key Finding: Variables like education and occupation
show strong predictive power for income even though
they can't be measured using simple correlation.


---
## Summary

In this assignment, we learned:

1. **Data Preprocessing:**
   - Handling missing values (imputation with mode)
   - Scaling: Standard (mean=0, std=1) vs Min-Max (range 0-1)

2. **Encoding:**
   - One-Hot: For categorical with few categories
   - Label: For categorical with many categories

3. **Feature Engineering:**
   - Created total_capital (net capital change)
   - Created age_group (categorized ages)
   - Applied log transformation to reduce skewness

4. **Feature Selection:**
   - Isolation Forest to detect and remove outliers
   - Correlation matrix for linear relationships
   - PPS for capturing all types of relationships